In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException
# Initialize the browser instance
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run Chrome in headless mode
driver = webdriver.Chrome(options=options)
# driver = webdriver.Chrome()

visited_links = set()
data = []
max_depth = 1

def fetch_content(url, depth=0):
    if depth > max_depth:
        return

    driver.get(url)
    visited_links.add(url)

    # Extract main content from the page (this might need refinement based on the website structure)
    # content = driver.find_element(By.TAG_NAME, 'body').text
    try:
        content = driver.find_element(By.TAG_NAME, 'body').text
    except StaleElementReferenceException:
        # Re-attempt to get the content
        content = driver.find_element(By.TAG_NAME, 'body').text

    data.append({
        'url': url,
        'content': content
    })

    # Extract links from the page
    try:
        links = [link.get_attribute('href') for link in driver.find_elements(By.TAG_NAME, 'a')]
    except StaleElementReferenceException:
        # Re-attempt to get the links
        links = [link.get_attribute('href') for link in driver.find_elements(By.TAG_NAME, 'a')]

    # Extract links from the page
    # links = [link.get_attribute('href') for link in driver.find_elements(By.TAG_NAME, 'a')]

    for link in links:
        print('links ' + link)
        if link not in visited_links and 'docs.akoya.com' in link:  # Ensuring we're only visiting internal links
            fetch_content(link, depth+1)

fetch_content('https://docs.akoya.com/')

driver.quit()

# Now, `data` contains the content for each page.
# You can serialize this data to JSON or another format for further use
data

links https://docs.akoya.com/#content
links https://docs.akoya.com/#content
links https://www.akoya.com/
links https://docs.akoya.com/
links https://docs.akoya.com/
links https://docs.akoya.com/docs
links https://docs.akoya.com/reference
links https://docs.akoya.com/changelog
links https://akoya.force.com/support
links https://recipient.ddp.akoya.com/login
links https://www.akoya.com/
links https://akoya.force.com/support
links https://recipient.ddp.akoya.com/login
links https://docs.akoya.com/
links https://docs.akoya.com/docs
links https://docs.akoya.com/reference
links https://docs.akoya.com/changelog
links https://docs.akoya.com/#akoya-documentation-center
links https://docs.akoya.com/docs/akoya-v2-release
links https://docs.akoya.com/docs/data-access-network
links https://docs.akoya.com/docs/how-to-work-with-akoya
links https://docs.akoya.com/docs/akoya-v2-release
links https://docs.akoya.com/docs/products
links https://docs.akoya.com/docs/account-info
links https://docs.akoya.com

[{'url': 'https://docs.akoya.com/',
  'content': 'Home\n🔗 Support\nAkoya login\nv2.1\nHome\nGuides\nAPI Docs\nChangelog\nSearch\nAkoya Documentation Center\n⭐ Getting started\nNew to Akoya? Start with this outline of first steps that includes links to more details and resources.\n📚 API documentation\nRefer to the documentation for the Akoya Token and Data Access API endpoints.\n🚀 Postman collection\nThe Postman collection, environment files, and a guide for testing.\nIntroduction\nAkoya API v2\nData Access Network\nHow to work with Akoya\nView More…\nAkoya products\nIntroduction: Akoya products\nAccount info\nBalances\nView More…\nTechnical guides\nData Recipient Hub v5.0.0\nMigrating to Akoya API v2\nOpenAPI\nView More…\nSandbox data\nIntroduction: Sandbox\nMikomo\nValidating with Sandbox\nReference\n📚 Glossary\nCORS\ncURL\nView More…\nAuthorization/Tokens\nGet authorization code\nToken\nRefresh token\nRevoke ID Token\nAkoya APIs v2\nAccount information\nBalances\nCustomer info\nView 

In [23]:
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
import time
from bs4 import BeautifulSoup

# Initialize the Chrome WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run Chrome in headless mode
driver = webdriver.Chrome(options=options)

visited_links = set()

def fetch_content(url, depth=0, max_depth=10):

    if depth > max_depth:
        return ""
    print("fetching content for url " + url)
    try:
        driver.get(url)
        time.sleep(1)  # Wait for JavaScript to run and render the content
    except:
        print("error retrieving url: " + url)
        return ""

    # Collect content while preserving headings and sections
    # page_content = driver.page_source  # This fetches the entire page source after JS execution

    # soup = BeautifulSoup(driver.page_source, 'html.parser')
    # for script_tag in soup.find_all('script'):
    #     script_tag.decompose()
    # page_content = str(soup)
    try:
        page_content = driver.find_element(By.TAG_NAME, 'body').text
    except StaleElementReferenceException:
        # Re-attempt to get the content
        page_content = driver.find_element(By.TAG_NAME, 'body').text

    # print('page content ===============================================')
    # print(page_content)

    # Fetch all links on the page
    links = []
    try:
        links = [link.get_attribute('href') for link in driver.find_elements(By.TAG_NAME, 'a')]
    except StaleElementReferenceException:
        # Handle the exception and re-fetch the elements
        links = [link.get_attribute('href') for link in driver.find_elements(By.TAG_NAME, 'a')]

    for link in links:
        if link not in visited_links and link.__contains__(url):
            visited_links.add(link)
            page_content += fetch_content(link, depth + 1, max_depth)  # Recursive call

    return page_content

content = fetch_content("https://docs.akoya.com/reference")

# Minimalist CSS styling for headers, sections, etc.
css_style = """
<style>
    body {
        font-family: Arial, sans-serif;
    }
    h1, h2, h3, h4, h5 {
        color: #333;
        margin-top: 20px;
    }
    p {
        margin-bottom: 15px;
    }
    a {
        color: #007BFF;
        text-decoration: none;
    }
</style>
"""

with open('akoya-reference.html', 'w', encoding='utf-8') as file:
    file.write("<html><head>" + css_style + "</head><body>" + content + "</body></html>")

driver.quit()


fetching content for url https://docs.akoya.com/reference
fetching content for url https://docs.akoya.com/reference/get-authorization-code#content
fetching content for url https://docs.akoya.com/reference
fetching content for url https://docs.akoya.com/reference/get-authorization-code
fetching content for url https://docs.akoya.com/reference/get-authorization-code#the-request-url
fetching content for url https://docs.akoya.com/reference/get-authorization-code#build-your-own-url
fetching content for url https://docs.akoya.com/reference/get-authorization-code#example-sandbox-url
fetching content for url https://docs.akoya.com/reference/get-authorization-code#
fetching content for url https://docs.akoya.com/reference/token
fetching content for url https://docs.akoya.com/reference/token#content
fetching content for url https://docs.akoya.com/reference/token-errors
fetching content for url https://docs.akoya.com/reference/token-errors#content
fetching content for url https://docs.akoya.com/

True